# Introduction

A time series project on NYSE (New York Stock Exchange) data is an exciting endeavor that can provide valuable insights into the dynamics of the stock market. The NYSE, being one of the largest and most liquid stock exchanges in the world, offers a vast amount of historical data that can be leveraged to analyze and forecast stock prices, trading volumes, and other market indicators. Time series analysis, a crucial aspect of data analysis, involves the use of statistical and machine learning techniques to extract meaningful patterns and trends from sequential data.

In this project, we will delve into the world of NYSE data, exploring its characteristics, and applying time series techniques to uncover hidden patterns and relationships. The goal of this project is to develop a comprehensive understanding of the NYSE data, identify trends and anomalies, and build predictive models that can inform investment decisions or improve trading strategies.

# About the Data

The NYSE data used in this project will consist of historical stock prices, trading volumes, and other relevant market indicators. The data will be sourced from a reliable financial database, such as Quandl or Alpha Vantage, which provide access to large repositories of financial data.

The dataset will likely include the following features:

* Date: The date of each trading day

* Open: The opening price of each stock

* High: The highest price of each stock during the trading day

* Low: The lowest price of each stock during the trading day

* Close: The closing price of each stock

* Volume: The total trading volume of each stock

* Adj Close: The adjusted closing price of each stock, accounting for dividends and splits

The dataset will be cleaned and preprocessed to handle missing values, outliers, and other data quality issues. The project will focus on a specific subset of NYSE stocks, such as the S&P 500 index, to ensure a manageable and representative sample of the market.



# LOADING LIBRARIES
We are loading the necessary libraries for data analysis, modeling, and visualization. These libraries will be used for data processing, model training, and evaluation.

In [2]:
pip install ISLP

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 985.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 34.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 3.3 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=98a6129f267a74178510cea304a4f1b4d9fcf79a79ac9db441f264f5d53812e1
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autogra

In [3]:
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import \
     (LinearRegression,
      LogisticRegression,
      Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import \
     (train_test_split,
      GridSearchCV)

### Torch-Specific Imports
There are a number of imports for `torch`. (These are not
included with `ISLP`, so must be installed separately.)
First we import the main library
and essential tools used to specify sequentially-structured networks.

In [4]:
import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset

There are several other helper packages for `torch`. For instance,
the `torchmetrics` package has utilities to compute
various metrics to evaluate performance when fitting
a model. The `torchinfo` package provides a useful
summary of the layers of a model. We use the `read_image()`
function when loading test images in Section 10.9.4.

In [5]:
from torchmetrics import (MeanAbsoluteError,
                          R2Score)
from torchinfo import summary

The package `pytorch_lightning` is a somewhat higher-level
interface to `torch` that simplifies the specification and
fitting of
models by reducing the amount of boilerplate code needed
(compared to using `torch` alone).

In [6]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

In order to reproduce results we use `seed_everything()`. We will also instruct `torch` to use deterministic algorithms
where possible.

In [7]:
from pytorch_lightning import seed_everything
seed_everything(0, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)

We will use several datasets shipped with `torchvision` for our
examples: a pretrained network for image classification,
as well as some transforms used for preprocessing.

In [8]:
from torchvision.io import read_image
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50,
                                ResNet50_Weights)
from torchvision.transforms import (Resize,
                                    Normalize,
                                    CenterCrop,
                                    ToTensor)

We have provided a few utilities in `ISLP` specifically for this lab.
The `SimpleDataModule` and `SimpleModule` are simple
versions of objects used in `pytorch_lightning`, the
high-level module for fitting `torch` models. Although more advanced
uses such as computing on graphical processing units (GPUs) and parallel data processing
are possible in this module, we will not be focusing much on these
in this lab. The `ErrorTracker` handles
collections of targets and predictions over each mini-batch
in the validation or test stage, allowing computation
of the metric over the entire validation or test data set.

In [9]:
from ISLP.torch import (SimpleDataModule,
                        SimpleModule,
                        ErrorTracker,
                        rec_num_workers)

In addition we have included some helper
functions to load the
`IMDb` database, as well as a lookup that maps integers
to particular keys in the database. We’ve included
a slightly modified copy of the preprocessed
`IMDb` data from `keras`, a separate package
for fitting deep learning models. This saves us significant
preprocessing and allows us to focus on specifying and fitting
the models themselves.

In [10]:
from ISLP.torch.imdb import (load_lookup,
                             load_tensor,
                             load_sparse,
                             load_sequential)

Finally, we introduce some utility imports  not directly related to
`torch`.
The `glob()` function from the `glob` module is used
to find all files matching wildcard characters, which we will use
in our example applying the `ResNet50` model
to some of our own images.
The `json` module will be used to load
a JSON file for looking up classes to identify the labels of the
pictures in the `ResNet50` example.

In [11]:
from glob import glob
import json

This yields approximately two- or three-fold  acceleration for each epoch.
We have protected this code block using `try:` and `except:`
clauses; if it works, we get the speedup, if it fails, nothing happens.

### Time Series Prediction
We now show how to fit the models in Section 10.5.2
for  time series prediction.
We first load and standardize the data.

**Data Preprocessing**

It appears that you have loaded the NYSE data and are in the process of preprocessing it for analysis. Here's a breakdown of the code:

Data Loading and Column Selection

applied the StandardScaler n to standardize the selected columns. Standardization is a common technique used to rescale numeric data to have a mean of 0 and a standard deviation of 1. This can help improve the performance of machine learning models.

In [12]:
NYSE = load_data('NYSE')
cols = ['DJ_return', 'log_volume', 'log_volatility']
X = pd.DataFrame(StandardScaler(
                     with_mean=True,
                     with_std=True).fit_transform(NYSE[cols]),
                 columns=NYSE[cols].columns,
                 index=NYSE.index)

Next we set up the lagged versions of the data, dropping
any rows with missing values using the `dropna()`  method.

Feature Engineering: Lagged Features

creating lagged features for  time series data. Lagged features are a common technique used in time series analysis to capture patterns and relationships in the data.

In [13]:
for lag in range(1, 6):
    for col in cols:
        newcol = np.zeros(X.shape[0]) * np.nan
        newcol[lag:] = X[col].values[:-lag]
        X.insert(len(X.columns), "{0}_{1}".format(col, lag), newcol)
X.insert(len(X.columns), 'train', NYSE['train'])
X = X.dropna()

Finally, we extract the response, training indicator, and drop the current day’s `DJ_return` and
`log_volatility` to predict only from previous day’s data.

In [14]:
Y, train = X['log_volume'], X['train']
X = X.drop(columns=['train'] + cols)
X.columns

Index(['DJ_return_1', 'log_volume_1', 'log_volatility_1', 'DJ_return_2',
       'log_volume_2', 'log_volatility_2', 'DJ_return_3', 'log_volume_3',
       'log_volatility_3', 'DJ_return_4', 'log_volume_4', 'log_volatility_4',
       'DJ_return_5', 'log_volume_5', 'log_volatility_5'],
      dtype='object')

We first fit a simple linear model and compute the $R^2$ on the test data using
the `score()`  method.

**Model Evaluation: Linear Regression**

In this section, we evaluate the performance of a linear regression model on our dataset.

**Model Training**

We trained a linear regression model using the LinearRegression class from scikit-learn. The model was fit to the training data X[train] and Y[train], where X is the feature matrix and Y is the target variable.

Model Evaluation

To evaluate the performance of the model, we used the score method, which returns the coefficient of determination (R-squared) of the prediction. We evaluated the model on the testing data X[~train] and Y[~train], where ~train denotes the complement of the training set.

In [15]:
M = LinearRegression()
M.fit(X[train], Y[train])
M.score(X[~train], Y[~train])

0.4128912938562521

We refit this model, including the factor variable `day_of_week`.
For a categorical series in `pandas`, we can form the indicators
using the `get_dummies()`  method.

In [16]:
X_day = pd.merge(X, 
                 pd.get_dummies(NYSE['day_of_week']),
                 on='date')

 Note that we do not have
to reinstantiate the linear regression model
as its `fit()` method accepts a design matrix and a response directly.

In [17]:
M.fit(X_day[train], Y[train])
M.score(X_day[~train], Y[~train])

0.4595563133053273

This model achieves an $R^2$ of about 46%.

# Observations

The R-squared value of 0.4595563133053273 indicates that the model explains approximately 46.00% of the variance in the target variable. This suggests that the model has some predictive power, but there is still room for improvement.


*Some possible reasons for the moderate performance of the model include:*


* The simplicity of the linear regression model, which may not capture complex relationships between the features and target variable.
* The quality of the features, which may not be informative enough to predict the target variable accurately.
* The presence of noise or outliers in the data, which can affect the model's performance.

**To improve the model's performance, we may consider:**

* Using more advanced machine learning models, such as decision trees or neural networks.
* Engineering additional features that capture more complex relationships between the variables.
* Implementing regularization techniques to reduce overfitting.

Collecting more data to increase the sample size and improve the model's generalizability.

To fit the RNN, we must reshape the data, as it will expect 5 lagged
versions of each feature as indicated by the  `input_shape` argument
to the layer `nn.RNN()`  below. We first
ensure the columns of  our data frame are such that a reshaped
matrix will have the variables correctly lagged. We use the
`reindex()`  method to do this.

For an input shape `(5,3)`, each row represents a lagged version of the three variables.
The `nn.RNN()` layer also expects the first row of each
observation to be earliest in time, so we must reverse the current order.  Hence we loop over
`range(5,0,-1)` below, which is
an example of using a  `slice()`  to index
iterable objects. The general notation is `start:end:step`.

In [18]:
ordered_cols = []
for lag in range(5,0,-1):
    for col in cols:
        ordered_cols.append('{0}_{1}'.format(col, lag))
X = X.reindex(columns=ordered_cols)
X.columns

Index(['DJ_return_5', 'log_volume_5', 'log_volatility_5', 'DJ_return_4',
       'log_volume_4', 'log_volatility_4', 'DJ_return_3', 'log_volume_3',
       'log_volatility_3', 'DJ_return_2', 'log_volume_2', 'log_volatility_2',
       'DJ_return_1', 'log_volume_1', 'log_volatility_1'],
      dtype='object')

We now reshape the data.

In [19]:
X_rnn = X.to_numpy().reshape((-1,5,3))
X_rnn.shape

(6046, 5, 3)

By specifying the first size as -1, `numpy.reshape()` deduces its size based on the remaining arguments.

Now we are ready to proceed with the RNN, which uses 12 hidden units, and  10%
dropout.
After passing through the RNN,  we extract the final time point as `val[:,-1]`
in `forward()` below. This gets passed through a 10% dropout and then flattened through
a linear layer.

## Model Architecture: NYSEModel
 
In this section, we describe the architecture of the NYSEModel, a neural network designed to predict stock prices based on historical data.

**Overview**

The NYSEModel is a simple recurrent neural network (RNN) that takes in a sequence of input data and outputs a predicted stock price. 

**The model consists of three main components:**

**RNN Layer:** The RNN layer is the core component of the model, responsible for processing the sequential input data. It has 3 input features, 12 hidden units, and uses the batch_first=True argument to indicate that the input data is batched.

**Dense Layer:** The dense layer is a fully connected layer that takes the output of the RNN layer and produces a single output value, representing the predicted stock price.

**Dropout Layer:** The dropout layer is used to regularize the model and prevent overfitting. It randomly drops out 10% of the neurons during training.

In [20]:
class NYSEModel(nn.Module):
    def __init__(self):
        super(NYSEModel, self).__init__()
        self.rnn = nn.RNN(3,
                          12,
                          batch_first=True)
        self.dense = nn.Linear(12, 1)
        self.dropout = nn.Dropout(0.1)
    def forward(self, x):
        val, h_n = self.rnn(x)
        val = self.dense(self.dropout(val[:,-1]))
        return torch.flatten(val)
nyse_model = NYSEModel()

In [32]:
datasets = []
for mask in [train, ~train]:
    X_rnn_t = torch.tensor(X_rnn[mask].astype(np.float32))
    Y_t = torch.tensor(Y[mask].astype(np.float32))
    datasets.append(TensorDataset(X_rnn_t, Y_t))
nyse_train, nyse_test = datasets

/tmp/ipykernel_30/2100042546.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y_t = torch.tensor(Y[mask].astype(np.float32))
/tmp/ipykernel_30/2100042546.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Y_t = torch.tensor(Y[mask].astype(np.float32))


Following our usual pattern, we inspect the summary.

In [33]:
summary(nyse_model,
        input_data=X_rnn_t,
        col_names=['input_size',
                   'output_size',
                   'num_params'])

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
NYSEModel                                [1770, 5, 3]              [1770]                    --
├─RNN: 1-1                               [1770, 5, 3]              [1770, 5, 12]             204
├─Dropout: 1-2                           [1770, 12]                [1770, 12]                --
├─Linear: 1-3                            [1770, 12]                [1770, 1]                 13
Total params: 217
Trainable params: 217
Non-trainable params: 0
Total mult-adds (M): 1.83
Input size (MB): 0.11
Forward/backward pass size (MB): 0.86
Params size (MB): 0.00
Estimated Total Size (MB): 0.97

We again put the two datasets into a data module, with a
batch size of 64.

configuring the data module in this way provides a flexible and efficient way to load and process the NYSE dataset, which can help to improve the performance and stability of the model.

In [34]:
max_num_workers = rec_num_workers()
nyse_dm = SimpleDataModule(nyse_train,
                           nyse_test,
                           num_workers=min(4, max_num_workers),
                           validation=nyse_test,
                           batch_size=64)

We run some data through our model to be sure the sizes match up correctly.

In [35]:
for idx, (x, y) in enumerate(nyse_dm.train_dataloader()):
    out = nyse_model(x)
    print(y.size(), out.size())
    if idx >= 2:
        break

torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])
torch.Size([64]) torch.Size([64])


We follow our previous example for setting up a trainer for a
regression problem, requesting the $R^2$ metric
to be be computed at each epoch.

# Cofiguration of model

In [36]:
nyse_optimizer = RMSprop(nyse_model.parameters(),
                         lr=0.001)
nyse_module = SimpleModule.regression(nyse_model,
                                      optimizer=nyse_optimizer,
                                      metrics={'r2':R2Score()})

Fitting the model should by now be familiar.
The results on the test data are very similar to the linear AR model.

# Traning Model

In [26]:
nyse_trainer = Trainer(deterministic=True,
                       max_epochs=200,
                       callbacks=[ErrorTracker()])
nyse_trainer.fit(nyse_module,
                 datamodule=nyse_dm)
nyse_trainer.test(nyse_module,
                  datamodule=nyse_dm)

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.6140583157539368     │
│          test_r2          │    0.41722869873046875    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.6140583157539368, 'test_r2': 0.41722869873046875}]

**Model Evaluation: Test Results

The model has been trained and evaluated on the test dataset, and the results are as follows:

**Test Loss:** The test loss is 0.6141, which indicates the average difference between the model's predictions and the actual values in the test dataset.

**Test R2:** The test R2 is 0.4172, which indicates the proportion of the variance in the test dataset that is explained by the model.

**Interpretation of Results**

The test loss and R2 values provide insight into the model's performance on unseen data. A lower test loss indicates that the model is making more accurate predictions, while a higher test R2 indicates that the model is explaining more of the variance in the data.

In this case, the test loss is relatively high, indicating that the model is not making extremely accurate predictions. However, the test R2 is moderate, indicating that the model is still explaining a significant proportion of the variance in the data.



In [37]:
datasets = []
for mask in [train, ~train]:
    X_day_t = torch.tensor(
                   np.asarray(X_day[mask]).astype(np.float32))
    Y_t = torch.tensor(np.asarray(Y[mask]).astype(np.float32))
    datasets.append(TensorDataset(X_day_t, Y_t))
day_train, day_test = datasets

Creating a data module follows a familiar pattern.

In [38]:
day_dm = SimpleDataModule(day_train,
                          day_test,
                          num_workers=min(4, max_num_workers),
                          validation=day_test,
                          batch_size=64)

We build a `NonLinearARModel()` that takes as input the 20 features and a hidden layer  with 32 units. The remaining steps are familiar.

**Key Changes from Previous Architecture**

The NonLinearARModel architecture differs from the previous architecture in several key ways:

Non-linear activation functions: The NonLinearARModel uses ReLU activation functions, which introduce non-linearity into the model.

**Dropout:** The NonLinearARModel uses dropout to regularize the model and prevent overfitting.

**Increased capacity:** The NonLinearARModel has a larger capacity than the previous architecture, with more layers and more parameters.

**Benefits of NonLinearARMode**l

The NonLinearARModel architecture provides several benefits, including:

Improved accuracy: The non-linear activation functions and increased capacity of the model allow it to capture more complex patterns in the data.

In [29]:
class NonLinearARModel(nn.Module):
    def __init__(self):
        super(NonLinearARModel, self).__init__()
        self._forward = nn.Sequential(nn.Flatten(),
                                      nn.Linear(20, 32),
                                      nn.ReLU(),
                                      nn.Dropout(0.5),
                                      nn.Linear(32, 1))
    def forward(self, x):
        return torch.flatten(self._forward(x))

In [30]:
nl_model = NonLinearARModel()
nl_optimizer = RMSprop(nl_model.parameters(),
                           lr=0.001)
nl_module = SimpleModule.regression(nl_model,
                                        optimizer=nl_optimizer,
                                        metrics={'r2':R2Score()})

We continue with the usual training steps, fit the model,
and evaluate the test error. We see the test $R^2$ is a slight improvement over the linear AR model that also includes `day_of_week`.

In [31]:
nl_trainer = Trainer(deterministic=True,
                         max_epochs=20,
                         callbacks=[ErrorTracker()])
nl_trainer.fit(nl_module, datamodule=day_dm)
nl_trainer.test(nl_module, datamodule=day_dm) 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.5608447790145874     │
│          test_r2          │    0.46773093938827515    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5608447790145874, 'test_r2': 0.46773093938827515}]

**Model Evaluation: NonLinearARModel Test Results**

The NonLinearARModel has been trained and evaluated on the test dataset, and the results are as follows:

**Test Loss:** The test loss is 0.5608, which indicates the average difference between the model's predictions and the actual values in the test dataset.

**Test R2:** The test R2 is 0.4677, which indicates the proportion of the variance in the test dataset that is explained by the model.

## Observations

The test results for the NonLinearARModel show a slight improvement in performance compared to the previous model. 
The test R2 has increased from 0.4172 to 0.4677, indicating that the model is explaining more of the variance in the data.

The test loss has also decreased from 0.6141 to 0.5608, indicating that the model is making more accurate predictions.